## ニューラルネットワークの実装（基礎）

In [54]:
# Step 1 : データセットを準備
# Step 2 : モデルを定義
# Step 3 : 目的関数を選択
# Step 4 : 最適化手法を選択
# Step 5 : モデルを学習

### データセットを準備

In [55]:
from sklearn.datasets import load_iris

# Iris データセットの読み込み
x, t = load_iris(return_X_y=True)

In [56]:
# 形の確認
x.shape, t.shape

((150, 4), (150,))

In [57]:
# 型の確認
type(x), type(t)

(numpy.ndarray, numpy.ndarray)

In [58]:
# データ型の確認
x.dtype, t.dtype

(dtype('float64'), dtype('int32'))

In [59]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

In [60]:
# データ型の変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [61]:
# 型の確認
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [62]:
# データ型の確認
x.dtype, t.dtype

(torch.float32, torch.int64)

In [63]:
# 入力変数と目的変数をまとめて、ひとつのオブジェクト dataset に変換
dataset = torch.utils.data.TensorDataset(x, t)
dataset

In [64]:
# 型の確認
type(dataset)

torch.utils.data.dataset.TensorDataset

In [65]:
# (入力値, 目標値) のようにタプルで格納されている
dataset[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

In [66]:
# 型の確認
type(dataset[0])

tuple

In [67]:
# 1 サンプル目の入力値
dataset[0][0]

tensor([5.1000, 3.5000, 1.4000, 0.2000])

In [68]:
# 1 サンプル目の目標値
dataset[0][1]

tensor(0)

In [69]:
# サンプル数は len で取得可能
len(dataset)

150

In [70]:
# 各データセットのサンプル数を決定
# train : val: test = 60%　: 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [71]:
# それぞれのサンプル数を確認
n_train, n_val, n_test

(90, 30, 30)

In [72]:
# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [73]:
# サンプル数の確認
len(train), len(val), len(test)

(90, 30, 30)

### ミニバッチ学習

In [74]:
# バッチサイズ
batch_size = 10

In [75]:
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

### モデルの定義

In [76]:
class Net(nn.Module):

    # 使用するオブジェクトを定義
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(4, 4)
        self.fc2 = nn.Linear(4, 3)
    
    # 順伝播
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

In [77]:
# 乱数のシードを固定して再現性を確保
torch.manual_seed(0)

# インスタンス化
net = Net()

In [78]:
# モデルの確認
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

### 目的関数を選択

In [79]:
# 目的関数の設定
criterion = F.cross_entropy
criterion

<function torch.nn.functional.cross_entropy(input: torch.Tensor, target: torch.Tensor, weight: Union[torch.Tensor, NoneType] = None, size_average: Union[bool, NoneType] = None, ignore_index: int = -100, reduce: Union[bool, NoneType] = None, reduction: str = 'mean', label_smoothing: float = 0.0) -> torch.Tensor>

### 最適化手法の選択

In [80]:
# net.parameters() を展開
for parameter in iter(net.parameters()):
    print(parameter)

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)
Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)
Parameter containing:
tensor([[ 0.1816,  0.4152, -0.1029,  0.3742],
        [-0.0806,  0.0529,  0.4527, -0.4638],
        [-0.3148, -0.1266, -0.1949,  0.4320]], requires_grad=True)
Parameter containing:
tensor([-0.3241, -0.2302, -0.3493], requires_grad=True)


In [81]:
# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [82]:
# バッチサイズ分のサンプルの抽出
batch = next(iter(train_loader))
batch

[tensor([[5.4000, 3.9000, 1.7000, 0.4000],
         [4.6000, 3.6000, 1.0000, 0.2000],
         [6.5000, 3.0000, 5.5000, 1.8000],
         [6.9000, 3.1000, 5.4000, 2.1000],
         [6.3000, 2.5000, 4.9000, 1.5000],
         [7.1000, 3.0000, 5.9000, 2.1000],
         [5.8000, 2.7000, 4.1000, 1.0000],
         [7.0000, 3.2000, 4.7000, 1.4000],
         [6.7000, 3.0000, 5.0000, 1.7000],
         [7.2000, 3.6000, 6.1000, 2.5000]]),
 tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])]

In [83]:
# 入力値と目標値に分割
x, t = batch

In [84]:
# 入力値の確認
x

tensor([[5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [6.5000, 3.0000, 5.5000, 1.8000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [6.3000, 2.5000, 4.9000, 1.5000],
        [7.1000, 3.0000, 5.9000, 2.1000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [7.0000, 3.2000, 4.7000, 1.4000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [7.2000, 3.6000, 6.1000, 2.5000]])

In [85]:
# 目標値の確認
t

tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])

In [86]:
# 全結合層 fc1 の重み
net.fc1.weight

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)

In [87]:
# 全結合層 fc1 のバイアス
net.fc1.bias

Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)

In [88]:
# 全結合層 fc2 の重み
net.fc2.weight

Parameter containing:
tensor([[ 0.1816,  0.4152, -0.1029,  0.3742],
        [-0.0806,  0.0529,  0.4527, -0.4638],
        [-0.3148, -0.1266, -0.1949,  0.4320]], requires_grad=True)

In [89]:
# 全結合層 fc2 のバイアス
net.fc2.bias

Parameter containing:
tensor([-0.3241, -0.2302, -0.3493], requires_grad=True)

In [90]:
# 予測値の算出
y = net.forward(x)
y

tensor([[-0.2557, -0.2605, -0.4679],
        [-0.2041, -0.2834, -0.5574],
        [-0.2786, -0.2244, -0.3632],
        [-0.2552, -0.2214, -0.3703],
        [-0.3241, -0.2302, -0.3493],
        [-0.2788, -0.2244, -0.3631],
        [-0.3241, -0.2302, -0.3493],
        [-0.3241, -0.2302, -0.3493],
        [-0.3090, -0.2282, -0.3539],
        [-0.1884, -0.2129, -0.3907]], grad_fn=<AddmmBackward0>)

In [91]:
# call メソッドを用いた forward の計算（推奨）
y = net(x)
y

tensor([[-0.2557, -0.2605, -0.4679],
        [-0.2041, -0.2834, -0.5574],
        [-0.2786, -0.2244, -0.3632],
        [-0.2552, -0.2214, -0.3703],
        [-0.3241, -0.2302, -0.3493],
        [-0.2788, -0.2244, -0.3631],
        [-0.3241, -0.2302, -0.3493],
        [-0.3241, -0.2302, -0.3493],
        [-0.3090, -0.2282, -0.3539],
        [-0.1884, -0.2129, -0.3907]], grad_fn=<AddmmBackward0>)

In [92]:
# 目的関数の計算 criterion の call メソッドを利用
loss = criterion(y, t)
loss

tensor(1.0882, grad_fn=<NllLossBackward0>)

In [93]:
# 全結合層 fc1 の重みに関する勾配
net.fc1.weight.grad

In [94]:
# 全結合層 fc1 のバイアスに関する勾配
net.fc1.bias.grad

In [95]:
# 全結合層 fc2 の重みに関する勾配
net.fc2.weight.grad

In [96]:
# 全結合層 fc2 のバイアスに関する勾配
net.fc2.bias.grad

In [97]:
# 勾配の算出
# net => y => loss のように計算を行うとその関係性も保存されており、
# 勾配の算出の際には逆向きに net <= y <= loss のように計算結果が格納されているため、
# net 内の属性が変化する
loss.backward()

In [98]:
# 全結合層 fc1 の重みに関する勾配
net.fc1.weight.grad

tensor([[-0.2311, -0.1731, -0.0627, -0.0139],
        [ 0.7327,  0.3358,  0.6025,  0.2229],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

In [99]:
# 全結合層 fc1 のバイアスに関する勾配
net.fc1.bias.grad

tensor([-0.0461,  0.1060,  0.0000,  0.0000])

In [100]:
# 全結合層 fc2 の重みに関する勾配
net.fc2.weight.grad

tensor([[-0.0652,  0.0259,  0.0000,  0.0000],
        [ 0.0366,  0.0227,  0.0000,  0.0000],
        [ 0.0285, -0.0486,  0.0000,  0.0000]])

In [101]:
# 全結合層 fc2 のバイアスに関する勾配
net.fc2.bias.grad

tensor([ 0.1415, -0.0452, -0.0963])

In [102]:
# 勾配の情報を用いたパラメータの更新
optimizer.step()

In [103]:
# 全結合層 fc1 の重み
net.fc1.weight

Parameter containing:
tensor([[ 0.0194,  0.2855, -0.4053, -0.3666],
        [-0.2659,  0.1005, -0.0702,  0.3742],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)

In [104]:
# 全結合層 fc1 のバイアス
net.fc1.bias

Parameter containing:
tensor([ 0.2023,  0.2894, -0.3390, -0.2177], requires_grad=True)

In [105]:
# 全結合層 fc2 の重み
net.fc2.weight

Parameter containing:
tensor([[ 0.1881,  0.4126, -0.1029,  0.3742],
        [-0.0843,  0.0506,  0.4527, -0.4638],
        [-0.3176, -0.1217, -0.1949,  0.4320]], requires_grad=True)

In [106]:
# 全結合層 fc2 のバイアス
net.fc2.bias

Parameter containing:
tensor([-0.3382, -0.2256, -0.3397], requires_grad=True)

In [107]:
# 演算に使用できる GPU の有無を確認
torch.cuda.is_available()

False

In [108]:
# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [109]:
# 指定したデバイスへのモデルの転送
net.to(device)

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [110]:
# 指定したデバイスへの入力変数の転送
x = x.to(device)
x

tensor([[5.4000, 3.9000, 1.7000, 0.4000],
        [4.6000, 3.6000, 1.0000, 0.2000],
        [6.5000, 3.0000, 5.5000, 1.8000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [6.3000, 2.5000, 4.9000, 1.5000],
        [7.1000, 3.0000, 5.9000, 2.1000],
        [5.8000, 2.7000, 4.1000, 1.0000],
        [7.0000, 3.2000, 4.7000, 1.4000],
        [6.7000, 3.0000, 5.0000, 1.7000],
        [7.2000, 3.6000, 6.1000, 2.5000]])

In [111]:
# 指定したデバイスへの目的変数の転送
t = t.to(device)
t

tensor([0, 0, 2, 2, 1, 2, 1, 1, 1, 2])

In [112]:
# 勾配情報の初期化
optimizer.zero_grad()

In [113]:
# 初期化後の勾配情報
net.fc1.weight.grad

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [114]:
net.fc1.bias.grad

tensor([0., 0., 0., 0.])

### モデルを学習

In [115]:
# エポックの数
max_epoch = 1

In [116]:
# モデルの初期化
torch.manual_seed(0)

# モデルのインスタンス化とデバイスへの転送
net = Net().to(device)

In [117]:
# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [118]:
# 学習ループ
for epoch in range(max_epoch):
    
    for batch in train_loader:
        
        # バッチサイズ分のサンプルを抽出
        x, t = batch
        
        # 学習時に使用するデバイスへデータの転送
        x = x.to(device)
        t = t.to(device)
        
        # パラメータの勾配を初期化
        optimizer.zero_grad()
        
        # 予測値の算出
        y = net(x)
        
        # 目標値と予測値から目的関数の値を算出
        loss = criterion(y, t)
        
        # 目的関数の値を表示して確認
        # item(): tensot.Tensor => float
        print('loss: ', loss.item())
        
        # 各パラメータの勾配を算出
        loss.backward()
        
        # 勾配の情報を用いたパラメータの更新
        optimizer.step()

loss:  1.0881630182266235
loss:  1.0393922328948975
loss:  1.002811312675476
loss:  1.0250868797302246
loss:  1.0088638067245483
loss:  0.9351975321769714
loss:  0.8939588665962219
loss:  0.9765418171882629
loss:  0.9651519060134888


In [119]:
y

tensor([[-0.2667, -0.2892, -0.4206],
        [-0.3219, -0.2649, -0.3529],
        [ 0.2932, -0.5355, -1.1068],
        [-0.3763, -0.2410, -0.2863],
        [-0.2852, -0.2811, -0.3979],
        [-0.3763, -0.2410, -0.2863],
        [ 0.2476, -0.5154, -1.0510],
        [-0.3763, -0.2410, -0.2863],
        [-0.2507, -0.2963, -0.4402],
        [-0.2224, -0.3087, -0.4749]], grad_fn=<AddmmBackward0>)

In [120]:
# dim=1 で行ごとの最大値に対する要素番号を取得（dim=0 は列ごと）
y_label = torch.argmax(y, dim=1)

In [121]:
# 予測値から最大となるクラスの番号を取り出した結果
y_label

tensor([0, 1, 0, 1, 1, 1, 0, 1, 0, 0])

In [122]:
# 目的変数
t

tensor([1, 1, 0, 2, 1, 2, 0, 2, 1, 1])

In [123]:
# 値が一致しているか確認
y_label == t

tensor([False,  True,  True, False,  True, False,  True, False, False, False])

In [124]:
# 値が True となる個数の総和
torch.sum(y_label == t)

tensor(4)

In [125]:
# int => float 
torch.sum(y_label == t) * 1.0

tensor(4.)

In [126]:
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc

tensor(0.4000)

In [127]:
# モデルの初期化
torch.manual_seed(0)

# モデルのインスタンス化とデバイスへの転送
net = Net().to(device)

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

In [128]:
for epoch in range(max_epoch):

    for batch in train_loader:

        x, t = batch
        x = x.to(device)
        t = t.to(device)
        optimizer.zero_grad()
        y = net(x)
        loss = criterion(y, t)

        # New：正解率の算出
        y_label = torch.argmax(y, dim=1)
        acc  = torch.sum(y_label == t) * 1.0 / len(t)
        print('accuracy:', acc)

        loss.backward()
        optimizer.step()

accuracy: tensor(0.6000)
accuracy: tensor(0.8000)
accuracy: tensor(0.6000)
accuracy: tensor(0.6000)
accuracy: tensor(0.6000)
accuracy: tensor(0.5000)
accuracy: tensor(0.4000)
accuracy: tensor(0.2000)
accuracy: tensor(0.4000)


### 学習後の正解率を検証

In [129]:
# 正解率の計算
def calc_acc(data_loader):
    
    with torch.no_grad():
        
        accs = [] # 各バッチごとの結果格納用
        
        for batch in data_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = net(x)
            
            y_label = torch.argmax(y, dim=1)
            acc = torch.sum(y_label == t) * 1.0 / len(t)
            accs.append(acc)
            
    # 全体の平均を算出
    avg_acc = torch.tensor(accs).mean()
    print('Accuracy: {:.1f}%'.format(avg_acc * 100))
    
    return avg_acc

In [130]:
# 検証データで確認
calc_acc(val_loader)

Accuracy: 63.3%


tensor(0.6333)

In [131]:
# テストデータで確認
calc_acc(test_loader)

Accuracy: 56.7%


tensor(0.5667)